# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [ ]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [ ]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

intermittent_df_result_all_params = Croston_output_all_params(sales_intermittent,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 13.138320790416666 minutes


In [ ]:
intermittent_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_1_001_CA_1_validation,0.0,0.633286,0.1
1,FOODS_1_001_CA_1_validation,0.0,0.633286,0.1
2,FOODS_1_001_CA_1_validation,0.0,0.633286,0.1
3,FOODS_1_001_CA_1_validation,2.0,0.633286,0.1
4,FOODS_1_001_CA_1_validation,0.0,0.614705,0.1
...,...,...,...,...
1192468,FOODS_3_826_CA_4_validation,4.0,0.395707,0.9
1192469,FOODS_3_826_CA_4_validation,4.0,3.144721,0.9
1192470,FOODS_3_826_CA_4_validation,1.0,3.900861,0.9
1192471,FOODS_3_826_CA_4_validation,2.0,1.294777,0.9


In [ ]:
intermittent_df_result_all_params.to_csv('Croston_Intermittent_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = intermittent_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.2677630991499995 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1_validation,0.1,0.879139,129.0799,157.2785,52.8595,0.976821,1.610662
1,FOODS_1_001_CA_1_validation,0.2,0.888063,130.3902,158.3659,53.7805,0.986736,1.619596
2,FOODS_1_001_CA_1_validation,0.3,0.907000,133.1706,157.7250,51.9402,1.007778,1.628162
3,FOODS_1_001_CA_1_validation,0.4,0.930992,136.6933,157.4308,51.2886,1.034436,1.639787
4,FOODS_1_001_CA_1_validation,0.5,0.960504,141.0264,157.5894,52.2765,1.067227,1.655770
...,...,...,...,...,...,...,...,...
32224,FOODS_3_826_CA_4_validation,0.5,0.946798,84.9530,88.6255,59.6444,1.446497,1.835858
32225,FOODS_3_826_CA_4_validation,0.6,0.980948,88.0171,89.2730,61.9506,1.498670,1.928204
32226,FOODS_3_826_CA_4_validation,0.7,1.017543,91.3007,90.1289,64.6961,1.554579,2.022128
32227,FOODS_3_826_CA_4_validation,0.8,1.047467,93.9857,90.6222,67.1208,1.600297,2.114494


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
90,FOODS_1_008_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
91,FOODS_1_008_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
92,FOODS_1_008_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
93,FOODS_1_008_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
94,FOODS_1_008_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
32215,FOODS_3_826_CA_3_validation,0.5,inf,inf,200.0,NaN,2.415989,2.415989
32216,FOODS_3_826_CA_3_validation,0.6,inf,inf,200.0,NaN,2.495264,2.495264
32217,FOODS_3_826_CA_3_validation,0.7,inf,inf,200.0,NaN,2.578302,2.578302
32218,FOODS_3_826_CA_3_validation,0.8,inf,inf,200.0,NaN,2.682351,2.682351


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
90,FOODS_1_008_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
91,FOODS_1_008_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
92,FOODS_1_008_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
93,FOODS_1_008_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
94,FOODS_1_008_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
32215,FOODS_3_826_CA_3_validation,0.5,inf,inf,200.0,NaN,2.415989,2.415989
32216,FOODS_3_826_CA_3_validation,0.6,inf,inf,200.0,NaN,2.495264,2.495264
32217,FOODS_3_826_CA_3_validation,0.7,inf,inf,200.0,NaN,2.578302,2.578302
32218,FOODS_3_826_CA_3_validation,0.8,inf,inf,200.0,NaN,2.682351,2.682351


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
90,FOODS_1_008_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
91,FOODS_1_008_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
92,FOODS_1_008_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
93,FOODS_1_008_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
94,FOODS_1_008_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
32125,FOODS_3_823_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
32126,FOODS_3_823_CA_1_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
32127,FOODS_3_823_CA_1_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
32128,FOODS_3_823_CA_1_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
90,FOODS_1_008_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
91,FOODS_1_008_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
92,FOODS_1_008_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
93,FOODS_1_008_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
94,FOODS_1_008_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
32215,FOODS_3_826_CA_3_validation,0.5,inf,inf,200.0,NaN,2.415989,2.415989
32216,FOODS_3_826_CA_3_validation,0.6,inf,inf,200.0,NaN,2.495264,2.495264
32217,FOODS_3_826_CA_3_validation,0.7,inf,inf,200.0,NaN,2.578302,2.578302
32218,FOODS_3_826_CA_3_validation,0.8,inf,inf,200.0,NaN,2.682351,2.682351


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 31.778832728288187 %
Percentage of unexpected values of WMAPE is: 31.778832728288187 %
Percentage of unexpected values of SMAPE is: 32.923764311644796 %
Percentage of unexpected values of MAPE is: 31.778832728288187 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,1.097864,148.959618,133.359482,56.823194,1.182068,1.516300
0.2,1.069518,142.856903,132.986079,57.702245,1.188109,1.522282
0.3,1.070019,141.605086,133.180938,58.780675,1.207684,1.544296
0.4,1.081392,142.247386,133.552683,60.004743,1.230959,1.572805
0.5,1.098645,143.918196,134.003453,61.311388,1.256811,1.606150
0.6,1.120764,146.415647,134.509623,62.731353,1.285609,1.644606
0.7,1.148638,149.851126,135.060523,64.338820,1.317998,1.689598
0.8,1.184150,154.504572,135.638387,66.157185,1.355326,1.744061
0.9,1.231550,161.030509,136.206643,68.214050,1.400108,1.813917


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.3
The optimum Alpha based on Mean SMAPE is: 0.2
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.863592,104.1636,131.0030,53.1936,0.912260,1.170465
0.2,0.873949,106.1114,130.5724,54.1483,0.918598,1.175068
0.3,0.889914,109.0606,130.6527,55.0833,0.935500,1.196044
0.4,0.908129,111.8440,131.3350,56.4444,0.959139,1.214890
0.5,0.926556,114.4365,131.5580,57.9318,0.980755,1.244236
0.6,0.946030,116.8191,132.4067,59.6840,1.009099,1.271093
0.7,0.966097,120.0017,132.9121,61.4636,1.037201,1.306593
0.8,0.987331,122.9059,133.4345,63.2273,1.062735,1.351871
0.9,1.016011,125.9607,134.1491,65.2009,1.104905,1.416524


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.2
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.2

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_intermittent, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.6538373240833342 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1_validation,0.773884,133.3462,151.9463,44.0492,0.952473,1.190942
1,FOODS_1_001_CA_4_validation,0.956934,154.5816,172.3559,65.0889,0.883323,1.370411
2,FOODS_1_002_CA_1_validation,0.665829,143.4093,199.2613,97.3461,0.307306,0.787324
3,FOODS_1_002_CA_3_validation,1.591224,171.3625,183.4116,75.9345,0.367205,0.420662
4,FOODS_1_002_CA_4_validation,0.813615,109.5251,159.7109,67.4038,0.938787,1.189356
...,...,...,...,...,...,...,...
3576,FOODS_3_825_CA_4_validation,2.889665,622.3894,190.4652,49.9133,0.444564,0.450893
3577,FOODS_3_826_CA_1_validation,1.123599,105.8776,115.4028,106.9940,2.420059,2.674396
3578,FOODS_3_826_CA_2_validation,0.940931,126.6638,118.4100,32.5809,0.723793,0.871040
3579,FOODS_3_826_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000


In [ ]:
df_result_final.isnull().sum()

Product       0
MASE       1087
WMAPE      1087
SMAPE      1108
MAPE       1307
MAE           0
RMSE          0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Intermittent_Test_Data.csv')